# Dimensionsreduzierung zur Identifikation von Autorenschaft: Einsatz der Principal Component Analysis

Die Principal Component Analysis (PCA), zu deutsch Hauptkomponentenanalyse, ist ein Algorithmus zur Dimensionsreduzierung, der neben anderen Einsatzgebieten zunehmend auch Einzug in die Analyse textueller Daten gefunden hat. [^fn1] Damit der Computer mit textuellen Daten rechnen kann, müssen diese in numerischen Werten repräsentiert sein. Textkorpora werden dazu etwa über die Auszählung von Worthäufigkeiten in einen Wortvektorraum übertragen: Für jedes Dokument eines Textkorpus wird ein Vektor erstellt, der beispielsweise Worthäufigkeiten der Wörter, die in dem Dokument enthalten sind, repräsentiert. Da Textkorpora viele Dokumente umfassen können und damit ein aus sehr vielen Wörtern bestehendes Vokabular bilden, besteht der Wortvektorraum aus Vektoren mit sehr vielen Dimensionen. Unter einer Dimension ist hier die Spalte einer Document-Term-Matrix (DTM) gemeint, die alle Häufigkeiten zu einem Wort des Vokabulars eines Korpus enthält. Die Zeilen der DTM enthalten die Vektoren mit den Worthäufigkeiten, der Wörter, die in einem Dokument enthalten sind. Allerdings lassen sich für einen menschlichen Betrachter nur zwei bzw. drei Dimensionen visualisieren. Um mehrere Dimensionen auf nur zwei oder drei Dimensionen zu bringen, können Verfahren der Dimensionsreduzierung angewandt werden. Eines dieser Verfahren ist die Principal Component Analysis, die nachfolgend veranschaulicht werden soll.

Die im Folgenden genutzten [Daten](https://doi.org/10.5281/zenodo.3560761) [^fn2] sowie der Code, der leicht angepasst verwendet wird, basiert konkret auf einem [Unterkapitel](https://www.humanitiesdataanalysis.org/stylometry/notebook.html#principal-component-analysis) sowie auf dem einführenden Kapitel [Stylometry and the Voice of Hildegard](https://www.humanitiesdataanalysis.org/stylometry/notebook.html#) aus der praktischen Einführung Humanities Data Analysis von Karsdorp, Kestemont und Riddell. [^fn3] Als historisches Fallbeispiel dienen mittelalterliche Quellen, die entweder von Hildegard von Bingen oder von einem ihrer Sekretäre verfasst wurden. Ein informatives Video zu diesem Fall steht [hier](https://vimeo.com/70881172) zur Verfügung. Das Video wurde von den Forschenden erstellt, die diese unklare Lage bei der Autorenschaft der lateinischen Quellen aus dem 12. Jahrhundert wissenschaftlich untersucht haben. [^fn4]

[^fn1]: Jonathon Shlens, A Tutorial on Principal Component Analysis, in: arXiv:1404.1100, (3. April 2014), http://arxiv.org/abs/1404.1100.

[^fn2]: Folgert Karsdorp, Mike Kestemont, Allen Roddell, Supplemental Materials for "Humanities Data Analysis" [Data set]. Zenodo, https://doi.org/10.5281/zenodo.3560761.

[^fn3]: Folgert Karsdorp, Mike Kestemont, Allen Roddell, Humanities Data Analysis. Case Studies with Python, Princeton 2021, S. 248-280, hier 252-254, https://www.humanitiesdataanalysis.org/index.html

[^fn4]: Mike Kestemont, Sara Moens, Jeroen Deploige, Collaborative authorship in the twelfth century: A stylometric study of Hildegard of Bingen and Guibert of Gembloux, in: Digital Scholarship in the Humanities 30, Nr. 2 (2015), S. 199–224, https://doi.org/10.1093/llc/fqt063


## Import

In [ ]:
import os

import sklearn.feature_extraction.text as text
import sklearn.preprocessing as preprocessing
import sklearn.decomposition

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Helferfunktion

### Zum Laden der Daten

Im Ordner 'texts' befinden sich drei Textkorpora: Zum einen von Hildegard von Bingen, von ihrem Sekretär Guibert von Gembloux sowie als unabhängige Vergleichsgröße von einem weiteren Zeitgenossen, Bernard von Clairvaux. Die lateinischen Textkorpora liegen bereits in lemmatisierter Form vor. Da zur stilometrischen Analyse die zu vergleichenden Texte eine einheitliche Länge aufweisen sollen, bringt die nachfolgende Funktion die Textkorpora auf den entsprechenden Umfang. Die Länge wird bestimmt vom kürzesten der zu untersuchenden Texte. Die im Unterordner 'test' abgelegten Dateien enthalten die zu prüfenden Quellen mit den unklaren Autor:innenzuschreibungen.

In [ ]:
def load_directory(directory, max_length):
    documents, authors, titles = [], [], []
    for filename in os.scandir(directory):
        if not filename.name.endswith('.txt'):
            continue
        author, _ = os.path.splitext(filename.name)

        with open(filename.path) as f:
            contents = f.read()
        lemmas = contents.lower().split()
        start_idx, end_idx, segm_cnt = 0, max_length, 1

        # extract slices from the text:
        while end_idx < len(lemmas):
            documents.append(' '.join(lemmas[start_idx:end_idx]))
            authors.append(author[0])
            title = filename.name.replace('.txt', '').split('_')[1]
            titles.append(f"{title}-{segm_cnt}")

            start_idx += max_length
            end_idx += max_length
            segm_cnt += 1

    return documents, authors, titles

### Laden der Daten

Die Textkorpora werden in Abschnitte (chunks) mit einer Länge von 10.000 Tokens gebracht. In den zurückgegebenen Listen befinden sich die Quellentexte als string (documents), der erste Buchstabe des:r Autor:in (authors) als Label für die spätere Klassifizierung und Visualisierung sowie der restliche Teil des Dateinamens (title).

In [ ]:
documents, authors, titles = load_directory('data/texts', 10000)

In [ ]:
print(f'Anzahl der Dokumente: {len(documents)}')
print(authors[:5])
print(titles[:5])

## Erstellen des Vokabulars

Computer verarbeiten Zahlen - daher müssen Texte in numerische Werte umgewandelt werden: Für jedes jedes Dokument in einem Textkorpus kann ein Vektor erstellt werden, der das Dokument repräsentiert. Bei stilometrischen Analysen wurde herausgefunden, dass Funktionswörter, insbesondere Stoppwörter, als starkes Signal zur Bestimmung von Autorenschaft herangezogen werden können. Funktionswörter werden von Autor:innen unbewusst in bestimmten Mustern genutzt, dadurch stellen sie eine sehr gute Signatur dar, um Autor:innen zu identifizieren. Für den vorliegenden Fall wurde eine Liste erstellt, die 65 Funktionswörter umfasst. Diese Liste wird zunächst aus der Datei wordlist.txt in die Variable vocab eingelesen. Dies ist das Vokabular, welches bei der weiteren stilometrischen Untersuchung zur Authorship Attribution genutzt wird. Mehr zum Hintergrund der Auswahl sowie zur Rolle von Funktionswörtern bietet ein [Unterkapitel](https://www.humanitiesdataanalysis.org/stylometry/notebook.html#function-words) in der Einführung Humanities Data Analysis von Karsdorp, Kestemont und Riddell.

In [ ]:
vocab = [l.strip() for l in open('data/wordlist.txt') if not l.startswith('#') and l.strip()][:65]

## Vektorisierung, Normalisierung und Standardisierung

Die Python-Bibliothek [scikit-learn](https://scikit-learn.org/stable/#) bietet viele Funktionalitäten und Algorithmen für die Datenanalyse sowie für Machine Learning. Die Objektklasse [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html?highlight=countvectorizer#sklearn.feature_extraction.text.CountVectorizer) ermöglicht es, die Häufigkeiten von Wörtern in einem Textkorpus zu ermitteln. Zugleich übernimmt sie auch die Tokenisierung von Texten, die hier im nachfolgenden Code über einen als Parameter übergebenen regulären Ausdruck erfolgt. Im Ergebnis erstellt der CountVectorizer auf der Basis des übergebenen Vokabulars eine Document-Term-Matrix mit Wordhäufigkeiten.

Beim Normalisieren wird jedes Element des Dokumentvektors durch die Länge des Vektors, d.h. durch die Summer aller Elemente des Vektors, geteilt. Zur Anwendung kommt wiederum eine [Methode zur Normalisierung](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html?highlight=normalize#sklearn.preprocessing.normalize) aus [scikit-learn](https://scikit-learn.org/stable/#), die im konkreten Fall mit L1 normalisiert, was der beschriebenen Division durch die Vektorlänge entspricht.

Eine Standardisierung der Werte in der Document-Term-Matrix übernimmt die Objektklasse [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html?highlight=standardscaler#sklearn.preprocessing.StandardScaler) aus scikit-learn.


In [ ]:
vectorizer = text.CountVectorizer(token_pattern=r"(?u)\b\w+\b", vocabulary=vocab)

v_documents = vectorizer.fit_transform(documents).toarray()
print(v_documents)

In [ ]:
v_documents = preprocessing.normalize(v_documents.astype(np.float64), 'l1')
scaler = preprocessing.StandardScaler()
print(v_documents)

In [ ]:
v_documents = scaler.fit_transform(v_documents)

print(f'N={v_documents.shape[0]} Dokumente mit '
      f'V={v_documents.shape[1]} Features (= Dimensionen).')

print(v_documents)

## Durchführen der PCA

Eine Dimensionsreduzierung mittels PCA ermöglicht es, ein Datenset mit vielen Dimensionen auf weniger Dimensionen zu bringen, wobei gleichzeitig die größtmögliche Annäherung an die ursprünglichen Daten erhalten bleibt. Es sollte daher klar sein, dass es sich bei den dimensionsreduzierten Daten nur um eine sich den Originaldaten annähernde Zusammenfassung handelt.

Die PCA ermittelt Korrelationen zwischen den Daten, um daraus neue Informationen zu generieren, indem korrelierenden Daten zusammengefasst werden. Betrachtet man Autor:innenstile, dann schließen sich beispielweise die Verwendung von bestimmten Artikeln und unbestimmten Artikeln aus: Neigt eine Person dazu, mehr bestimmte Artikel zu nutzen, dann sind in Texten weniger unbestimmte Artikel zu finden. Aus dieser Information lässt sich eine neue zusammengefasste Information gewinnen und zwei Dimensionen werden zu einer zusammengefasst. 

Mit zwei Zeilen Code werden die 65 Dimensionen, die dem Vokabular mit den 65 ausgewählten Funktionswörtern entspricht, auf zwei Dimensionen reduziert. Zum Einsatz kommt die Objektklasse [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html?highlight=pca#sklearn.decomposition.PCA) aus [scikit-learn](https://scikit-learn.org/stable/#), die die notwendigen Berechnungen durchführt. Der Parameter *n_components* gibt die Anzahl der Dimensionen an, auf die der Algorithmus die Daten reduzieren soll. Die reduzierte Document Term Matrix umfasst zwei Hauptkomponenten (principal components).

In [ ]:
pca = sklearn.decomposition.PCA(n_components=2)
documents_proj = pca.fit_transform(v_documents)

print(f'Die ursprüngliche DTM mit \n {v_documents.shape[0]} Dokumenten umfasst {v_documents.shape[1]} Dimensionen.\n')
print(f'Die dimensionsreduzierte DTM mit \n {documents_proj.shape[0]} Dokumenten umfasst {documents_proj.shape[1]} Dimensionen.')


In [ ]:
print('ursprüngliche DTM:')
print(v_documents[:2])
print('\ndimensionsreduzierte DTM:')
print(documents_proj[:2])

## Scatterplot der dimensionsreduzierten Dokument-Term-Matrix

Die zweidimensionalen Daten können in einem Koordinatensystem (scatterplot) visualisiert werden. Die erste Hauptkomponente wird auf der x-Achse, die zweite Hauptkomponente auf der y-Achse abgebildet. In der resultierenden Visualisierung bilden die Datenpunkte der dimensionsreduzierten Texte sehr deutliche Cluster. Auf diese Weise werden unterschiedliche Schreibstile der Autor:innen sichtbar.

### Helferfunktion zum Erstellen eines Plots

In [ ]:
def plot_pca(document_proj, var_exp, labels):
    
    fig, ax = plt.subplots(figsize=(8, 8))
    x1, x2 = documents_proj[:, 0], documents_proj[:, 1]
    ax.scatter(x1, x2, facecolors='none')
                    
    for p1, p2, author in zip(x1, x2, labels):
        color = 'red' if author not in ('H', 'G', 'B') else 'black'
        ax.text(p1, p2, 
                     author, 
                     ha='center',
                     color=color, 
                     va='center', 
                     fontsize=12)

    # add variance information to the axis labels:
    ax.set_xlabel(f'PC1 ({var_exp[0] * 100:.2f}%)')
    ax.set_ylabel(f'PC2 ({var_exp[1] * 100:.2f}%)')
    ax.set_xlim(left=None,right=11)
    ax.set_ylim(bottom=None, top=8)

    ax.set_title('Visualisierung der dimensionsreduzierten Textdaten')

In [ ]:
pca = sklearn.decomposition.PCA(n_components=2)
documents_proj = pca.fit_transform(v_documents)
var_exp = pca.explained_variance_ratio_

plot_pca(documents_proj, var_exp, authors)

## Durchschnittlich erklärte Varianz

Nach Durchführung einer PCA lässt sich angeben, welche Hauptkomponente wie genau eine Zusammenfassung der Originaldaten anbietet, wenn man jede Hauptkomponente für sich betrachtet. Die Hauptkomponenten werden danach geordnet, wie gut sie sich den Originaldaten annähern. Über die Methode *explained_variance_ratio_* können die entsprechenden Werte abgerufen und visualisiert werden. Dazu wird eine neue PCA durchgeführt allerdings ist nun die Anzahl der zu reduzierenden Dimensionen mit *n_components* gleich 36 angegeben, der Anzahl der vorhandenen Dokumente. Visualisiert wird, wie gut jede Hauptkomponente für sich die Originaldaten abbildet und wie gut die Hauptkomponenten diese kumulativ zusammenfassen. 

In [ ]:
pca = sklearn.decomposition.PCA(n_components=36)
pca.fit(v_documents)

var_exp = pca.explained_variance_ratio_
cum_var_exp = np.cumsum(var_exp)

fig, ax = plt.subplots(figsize=(10,5))

ax.bar(range(36), var_exp, alpha=0.5, align='center',
        label='indivuduell erklärte Varianz')

ax.step(range(36), cum_var_exp, where='mid',
         label='kumulativ erklärte Varianz')

ax.axhline(0.05, ls='dotted', color="black")
ax.set(ylabel='Durchschnittlich erklärte Varianz', xlabel='Hauptkomponente')
ax.legend(loc='center right')
ax.set_title('Veranschaulichung zur PCA');

## Einlesen und Verarbeiten der Testdaten

Um ein Befund für eine Authorship Attribution zu generieren, müssen nun die zu untersuchenden, unklaren Textdaten dem Korpus der bekannten Dokumente hinzugefügt werden.

In [ ]:
test_docs, test_authors, test_titles = load_directory('data/texts/test', 3301)

v_test_docs = vectorizer.transform(test_docs)
v_test_docs = preprocessing.normalize(v_test_docs.astype(float), norm='l1')
v_test_docs = scaler.transform(v_test_docs.toarray())

## Visualisieren der bekannten und unklaren Quellendokumente in einem Scatterplot

In [ ]:
all_documents = preprocessing.scale(np.vstack((v_documents, v_test_docs)))
pca = sklearn.decomposition.PCA(n_components=2)
documents_proj = pca.fit_transform(all_documents)
var_exp = pca.explained_variance_ratio_

plot_pca(documents_proj, var_exp, list(authors) + test_titles)

## Ergebnis

In der Visualisierung der dimensionsreduzierte Textdaten stehen  die Quellendokumente mit einer unklaren Autor:innenzuschreibung dem Cluster von Guibert von Gembloux nahe. Dessen Autorenschaft ist somit wahrscheinlicher als eine Autorenschaft von Hildegard von Bingen